In [74]:

import pandas as pd
import sys
import os
import importlib

# Lấy thư mục gốc project (thư mục chứa "services")
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

# Reload module để chắc chắn lấy file mới
import services.query_service.extract_category as extractSport
import services.query_service.extract_location as extract_location
import services.query_service.extract_time as extract_time
importlib.reload(extract_location)
importlib.reload(extractSport)
importlib.reload(extract_time)
from services.query_service.extract_location import ExtractLocation
from services.query_service.extract_category import ExtractSport
from services.query_service.extract_time import ExtractTime

query = "sân Cầu lông ở quận 9 buổi chiều"

parsed_df = pd.DataFrame([{"query": query}])
    # 1. Parse user query
parsed_df = ExtractLocation.merge_query_with_locations(parsed_df)
parsed_df = ExtractSport.extract_category_from_df(parsed_df)
parsed_df = ExtractTime.extract_time_from_df(parsed_df)

# print(ExtractLocation.extract_locations_to_dataframe(query.lower()))
print(parsed_df)

                              query phuong district         city  \
0  sân Cầu lông ở quận 9 buổi chiều         Thủ Đức  Hồ Chí Minh   

                               category       time  
0  a1f3b6e4-2c9a-4c1b-b8a2-9c0d4fe71201  afternoon  


In [75]:
import sys
import importlib

# Xóa module khỏi cache
sys.modules.pop('services.parse_user_query', None)

<module 'services.parse_user_query' from 'c:\\Users\\giaba\\source\\repos\\4\\Ky2\\booking_ground\\notebooks\\services\\parse_user_query.py'>

In [76]:
# Reload module
import services.parse_user_query as parse_module
importlib.reload(parse_module)

from services.parse_user_query import ParseUtils

In [81]:
# Cell 1: Import libraries
import pandas as pd
import numpy as np

def create_training_data(venues_df, historical_queries):
    training_data = []
    if not isinstance(historical_queries, pd.DataFrame):
        historical_queries = pd.DataFrame(historical_queries)


    for idx, query_record in historical_queries.iterrows():
        query = query_record["query"]
        clicked_venue_id = query_record.get("clicked_venue_id") or query_record.get("venueId")
        for _, venue_row in venues_df.iterrows():   
            features = ParseUtils.pair_feature(parsed_query=query_record, venue_row=venue_row)
            venue_id = venue_row.get("venueId") or venue_row.get("venueid") or venue_row.get("id")
            label = 1 if str(venue_id) == str(clicked_venue_id) else 0
            record = {
                "sport_match": features['sport_match'],
                "location_match" : features['location_match'],
                "time_match": features['time_match'],
                "label": label,
                "query": query,
                "venue_id": venue_id,
            }

            training_data.append(record)

    df_training = pd.DataFrame(training_data)
    return df_training

In [82]:
df = pd.read_csv('../datasets/df.csv')
df = df.drop(columns=["Unnamed: 0"])
df = df.drop_duplicates()
print(df)

     venueId      phuong      district         city  \
0      v-001    Tân Hưng             7  Hồ Chí Minh   
3      v-002  Phước Kiển        Nhà Bè  Hồ Chí Minh   
8      v-003          16             8  Hồ Chí Minh   
9      v-003          16             8  Hồ Chí Minh   
12     v-004          15      Tân Bình  Hồ Chí Minh   
...      ...         ...           ...          ...   
993    v-244     Láng Hạ       Đống Đa       Hà Nội   
998    v-245   Tân Triều     Thanh Trì       Hà Nội   
1003   v-246   Định Công     Hoàng Mai       Hà Nội   
1008   v-247   Bạch Đằng  Hai Bà Trưng       Hà Nội   
1013   v-248   Dịch Vọng      Cầu Giấy       Hà Nội   

                                categoryId start_time end_time  
0     a1f3b6e4-2c9a-4c1b-b8a2-9c0d4fe71201      05:00    22:00  
3     a1f3b6e4-2c9a-4c1b-b8a2-9c0d4fe71201      05:00    22:00  
8     e3a7b9d9-5c2f-4d33-b4c4-33f1d765cd04      05:00    22:00  
9     a1f3b6e4-2c9a-4c1b-b8a2-9c0d4fe71201      05:00    22:00  
12    a1f3b6e4

In [89]:
df_test = create_training_data (venues_df=df, historical_queries= parsed_df)
print(df_test)
df_test.to_csv('../datasets/df_test.csv')

     sport_match  location_match  time_match  label  \
0              1               1           1      0   
1              1               1           1      0   
2              0               1           1      0   
3              1               1           1      0   
4              1               1           1      0   
..           ...             ...         ...    ...   
298            0               0           1      0   
299            0               0           1      0   
300            1               0           1      0   
301            0               0           1      0   
302            0               0           1      0   

                                query venue_id  
0    sân Cầu lông ở quận 9 buổi chiều    v-001  
1    sân Cầu lông ở quận 9 buổi chiều    v-002  
2    sân Cầu lông ở quận 9 buổi chiều    v-003  
3    sân Cầu lông ở quận 9 buổi chiều    v-003  
4    sân Cầu lông ở quận 9 buổi chiều    v-004  
..                                ...      ..

In [97]:
import joblib

df_test = df_test[[ "sport_match", "location_match", "time_match"]]

print(df_test)

     sport_match  location_match  time_match
0              1               1           1
1              1               1           1
2              0               1           1
3              1               1           1
4              1               1           1
..           ...             ...         ...
298            0               0           1
299            0               0           1
300            1               0           1
301            0               0           1
302            0               0           1

[303 rows x 3 columns]


In [98]:
model = joblib.load("../checkpoints/dummy.pkl")
prediction = model.predict(df_test)


In [99]:
df_test["pred"] = model.predict(df_test)

In [100]:
print(df_test)

     sport_match  location_match  time_match  pred
0              1               1           1     0
1              1               1           1     0
2              0               1           1     0
3              1               1           1     0
4              1               1           1     0
..           ...             ...         ...   ...
298            0               0           1     0
299            0               0           1     0
300            1               0           1     0
301            0               0           1     0
302            0               0           1     0

[303 rows x 4 columns]
